In [ ]:
from matplotlib import *
from __future__ import division
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp


## homogeneous population, deterministic model

In [ ]:
# set the seeding city
seed = 0
nseeds = 10

# Parameters
num_patches = 5 # Number of patches
beta = 0.5     # Infection rate
gamma = 0.2      # Recovery rate
population = 400  # Total population per patch

# Time span for simulation
t_max = 200
t_span = (0, t_max)
t_eval = np.linspace(0, t_max, t_max*3)
dt = t_eval[1]-t_eval[0]

subpopulation_R0 = beta/gamma
attack_ratio = population*(1 - np.exp(-subpopulation_R0))
print(subpopulation_R0 , attack_ratio)

In [ ]:
#create random matrix of transitions 
np.random.seed(9001)
#let movements between i!=j must be lower than between i=j
OD_matrix = np.random.random_integers(low=0, high=0.1 * population / num_patches, size=(num_patches,num_patches))
#set diagonal to zero
n = OD_matrix.shape[0]
OD_matrix[range(n), range(n)] = 0 
#flows between i and j must be symmetric, those who go also come back
OD_matrix = (OD_matrix+OD_matrix.T)
#count how many do not move in each population
staying = population - OD_matrix.sum(axis=1)
#set movements in the diagonal i = j
OD_matrix[range(n), range(n)] = staying
#normalize rows to sum to 1, these are rates of transition per population
row_sums = OD_matrix.sum(axis=1, keepdims=True)
# Transition matrix for mobility between patches (Markovian)
P = OD_matrix / row_sums

In [ ]:
staying

In [ ]:
OD_matrix

In [ ]:
OD_matrix.sum(axis=1, keepdims=True)

In [ ]:
P

In [ ]:
from scipy.stats import binom
binom.rvs(100, .2)

In [ ]:
# Force of infection function
def force_of_infection(beta, I, P, N):
    return ...

In [ ]:
# Initial conditions (S, I, R for each patch)
S0 = np.linspace(population,population, num_patches,dtype=int) # initial susceptible populations
S0[seed] -= nseeds   # remove seeds from seed susceptible populations

I0 = np.zeros(num_patches,dtype=int) # initial infected populations
I0[seed] = nseeds          # seeds
R0 = np.zeros(num_patches,dtype=int) # initial recovered populations

    
#define matrix of results
y = np.zeros((num_patches*3, len(t_eval)),dtype=int)

y[:num_patches,0] = S0 #top rows
y[num_patches:2*num_patches,0] = I0 #middle rows
y[2*num_patches:,0] = R0 #bottom rows

N = np.array([population] * num_patches)
# Matrix to store transitions
movementsS = np.zeros_like(P, dtype=int)
movementsI = np.zeros_like(P, dtype=int)
movementsR = np.zeros_like(P, dtype=int)


for t, tempo in enumerate(t_eval[:-1]):
    #reset new entries for states compartments
    nS, nI, nR = np.zeros(num_patches),np.zeros(num_patches),np.zeros(num_patches)

    # Reshape the state vector y into S, I, R for each patch
    S = y[:num_patches,t] #top rows
    I = y[num_patches:2*num_patches,t] #middle rows
    R = y[2*num_patches:,t] #bottom rows
    
    # Calculate the force of infection for each patch
    lambda_i = force_of_infection(beta, I, P, N)
    
    # Compute the derivatives for each patch
    # Reaction
    new_infected = binom.rvs(S,lambda_i*dt) 
    new_recovered = binom.rvs(I,gamma*dt) 
    
    # Diffusion
    for i in range(num_patches):
        for j in range(len(S0)):
            if i != j:  # No self-transition
                movementsS[i, j] = binom.rvs(S[i] - new_infected[i], P[i, j]*dt)
                movementsI[i, j] = binom.rvs(I[i] + new_infected[i] - new_recovered[i], P[i, j]*dt)
                movementsR[i, j] = binom.rvs(R[i] + new_recovered[i], P[i, j]*dt)
                
    # Compute compartments variations in each population
    ns = - new_infected - movementsS.sum(axis=1) + movementsS.sum(axis=0)
    ni = ...
    nr = ...
    
    # update lists
    y[:num_patches,t+1] += S + ns
    y[num_patches:2*num_patches,t+1] += I + ni
    y[2*num_patches:,t+1] += R + nr


# Extract results
S, I, R = y[:num_patches,:], y[num_patches:2*num_patches,:], y[2*num_patches:,:]


In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))

# Plot Susceptible, Infected, and Recovered over time
for i in range(num_patches):
    plt.plot(t_eval, I[i], label=f'I{ i+1 } patch')
    plt.plot(t_eval, R[i], label=f'R{ i+1 } patch')
    #plt.plot(t_eval, S[i]+I[i]+R[i], label=f'R{ i+1 } (Recovered)')

plt.xlabel('Time')
plt.axhline(attack_ratio,ls='--', color='grey')
plt.ylim(0,population+10)
plt.ylabel('Active infected')
plt.legend(frameon=False)
plt.title('Metapopulation SIR Model with Markovian Mobility')
plt.grid(True)
plt.show()

In [ ]:
final_attack_ratio = R[:,-1]/population
print(final_attack_ratio*100, population)

## heterogeneous populations

In [ ]:
population = np.array([400,100,400,1000,400])  # Total population per patch
S0 = np.array(population, dtype=int)

#create random matrix of transitions 
np.random.seed(9001)
#let movements between i!=j must be lower than between i=j
OD_matrix = np.random.random_integers(low=0, high=0.5 * min(population) / num_patches, size=(num_patches,num_patches))
#set diagonal to zero
n = OD_matrix.shape[0]
OD_matrix[range(n), range(n)] = 0 
#flows between i and j must be symmetric, those who go also come back
OD_matrix = (OD_matrix+OD_matrix.T)
#count how many do not move in each population
staying = population - OD_matrix.sum(axis=1)
#set movements in the diagonal i = j
OD_matrix[range(n), range(n)] = staying
#normalize rows to sum to 1, these are rates of transition per population
row_sums = OD_matrix.sum(axis=1, keepdims=True)
# Transition matrix for mobility between patches (Markovian)
P = OD_matrix / row_sums

In [ ]:
staying

In [ ]:
OD_matrix

In [ ]:
OD_matrix.sum(axis=1, keepdims=True)

In [ ]:
P

In [ ]:

subpopulation_R0 = beta/gamma
attack_ratio = population*(1 - np.exp(-subpopulation_R0))
print(subpopulation_R0 , attack_ratio)

# Initial conditions (S, I, R for each patch)
S0 = np.array(population, dtype=int) # initial susceptible populations
S0[seed] -= nseeds   # remove seeds from seed susceptible populations

I0 = np.zeros(num_patches, dtype=int) # initial infected populations
I0[seed] = nseeds          # seeds
R0 = np.zeros(num_patches, dtype=int) # initial recovered populations

    
#define matrix of results
y = np.zeros((num_patches*3, len(t_eval)),dtype=int)

y[:num_patches,0] = S0 #top rows
y[num_patches:2*num_patches,0] = I0 #middle rows
y[2*num_patches:,0] = R0 #bottom rows

N = np.array([population] * num_patches, dtype=int)

for t, tempo in enumerate(t_eval[:-1]):
    #reset new entries for states compartments
    nS, nI, nR = np.zeros(num_patches,dtype=int),np.zeros(num_patches,dtype=int),np.zeros(num_patches,dtype=int)

    # Reshape the state vector y into S, I, R for each patch
    S = y[:num_patches,t] #top rows
    I = y[num_patches:2*num_patches,t] #middle rows
    R = y[2*num_patches:,t] #bottom rows
    
    # Calculate the force of infection for each patch
    lambda_i = force_of_infection(beta, I, P, population)
    
    # Compute the derivatives for each patch
    # Reaction
    new_infected = binom.rvs(S,lambda_i*dt) 
    new_recovered = binom.rvs(I,gamma*dt) 
    
    # Diffusion
    for i in range(num_patches):
        for j in range(len(S0)):
            if i != j:  # No self-transition
                movementsS[i, j] = binom.rvs(S[i] - new_infected[i], P[i, j]*dt)
                movementsI[i, j] = binom.rvs(I[i] + new_infected[i] - new_recovered[i], P[i, j]*dt)
                movementsR[i, j] = binom.rvs(R[i] + new_recovered[i], P[i, j]*dt)
                
    # Compute new population counts
    ...
    ...
    ...
    
    # update lists
    y[:num_patches,t+1] += S + ...
    y[num_patches:2*num_patches,t+1] += I + ...
    y[2*num_patches:,t+1] += R + ...


# Extract results
S, I, R = y[:num_patches,:], y[num_patches:2*num_patches,:], y[2*num_patches:,:]



In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))

# Plot Susceptible, Infected, and Recovered over time
for i in range(num_patches):
    plt.plot(t_eval, I[i], label=f'I{ i+1 } patch')
    #plt.plot(t_eval, S[i]+I[i]+R[i], label=f'R{ i+1 } (Recovered)')

plt.xlabel('Time')
plt.ylabel('Active infected')
plt.legend(frameon=False)
plt.title('Metapopulation SIR Model with Markovian Mobility')
plt.grid(True)
plt.show()

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))

# Plot Susceptible, Infected, and Recovered over time
for i in range(num_patches):
    plt.plot(t_eval, I[i], label=f'I{ i+1 } patch')
    plt.plot(t_eval, R[i], label=f'R{ i+1 } patch')
    plt.axhline(attack_ratio[i],ls='--', color='grey')

    #plt.plot(solution.t, S[i]+I[i]+R[i], label=f'R{ i+1 } (Recovered)')

plt.xlabel('Time')

plt.ylim(0,max(population))
plt.ylabel('Active infected')
plt.legend(frameon=False)
plt.title('Metapopulation SIR Model with Markovian Mobility')
plt.grid(True)
plt.show()

In [ ]:
final_attack_ratio = R[:,-1]/population
print(final_attack_ratio*100, population)

### apply travel bans

In [ ]:

ban = 0.5 

OD_matrix_ban = np.zeros((num_patches, num_patches))

for i in range(n):
    for j in range(n):
        if i!=j:
            OD_matrix_ban[i,j] = ...

#count how many do not move in each population
staying = population - OD_matrix_ban.sum(axis=1)
#set movements in the diagonal i = j
OD_matrix_ban[range(n), range(n)] = staying
            
#normalize rows to sum to 1, these are rates of transition per population
row_sums = OD_matrix_ban.sum(axis=1, keepdims=True)
# Transition matrix for mobility between patches (Markovian)
P = OD_matrix_ban / row_sums

In [ ]:
P

In [ ]:

subpopulation_R0 = beta/gamma
attack_ratio = population*(1 - np.exp(-subpopulation_R0))
print(subpopulation_R0 , attack_ratio)

# Initial conditions (S, I, R for each patch)
S0 = np.array(population, dtype=int) # initial susceptible populations
S0[seed] -= nseeds   # remove seeds from seed susceptible populations

I0 = np.zeros(num_patches, dtype=int) # initial infected populations
I0[seed] = nseeds          # seeds
R0 = np.zeros(num_patches, dtype=int) # initial recovered populations

    
#define matrix of results
y = np.zeros((num_patches*3, len(t_eval)),dtype=int)

y[:num_patches,0] = S0 #top rows
y[num_patches:2*num_patches,0] = I0 #middle rows
y[2*num_patches:,0] = R0 #bottom rows

N = np.array([population] * num_patches, dtype=int)

for t, tempo in enumerate(t_eval[:-1]):
    #reset new entries for states compartments
    nS, nI, nR = np.zeros(num_patches,dtype=int),np.zeros(num_patches,dtype=int),np.zeros(num_patches,dtype=int)

    # Reshape the state vector y into S, I, R for each patch
    Sc = y[:num_patches,t] #top rows
    Ic = y[num_patches:2*num_patches,t] #middle rows
    Rc = y[2*num_patches:,t] #bottom rows
    
    # Calculate the force of infection for each patch
    lambda_i = force_of_infection(beta, Ic, P, population)
    # Compute the derivatives for each patch
    # Reaction
    new_infected = binom.rvs(Sc,lambda_i*dt) 
    new_recovered = binom.rvs(Ic,gamma*dt) 
    
    # Diffusion
    for i in range(num_patches):
        for j in range(len(S0)):
            if i != j:  # No self-transition
                movementsS[i, j] = binom.rvs(Sc[i] - new_infected[i], P[i, j]*dt)
                movementsI[i, j] = binom.rvs(Ic[i] + new_infected[i] - new_recovered[i], P[i, j]*dt)
                movementsR[i, j] = binom.rvs(Rc[i] + new_recovered[i], P[i, j]*dt)
                
    # Compute new population counts
    ...
    ...
    ...
    
    # update lists
    y[:num_patches,t+1] += Sc + ns
    y[num_patches:2*num_patches,t+1] += Ic + ni
    y[2*num_patches:,t+1] += Rc + nr


# Extract results
Sc, Ic, Rc = y[:num_patches,:], y[num_patches:2*num_patches,:], y[2*num_patches:,:]



In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))

# Plot Susceptible, Infected, and Recovered over time
for i in range(num_patches):
    plt.plot(t_eval, I[i], label=f'I{ i+1 } patch')
    plt.plot(t_eval, Ic[i], label=f'Ic{ i+1 } patch')

    #plt.plot(solution.t, S[i]+I[i]+R[i], label=f'R{ i+1 } (Recovered)')

plt.xlabel('Time')


plt.ylabel('Active infected')
plt.legend(frameon=False)
plt.title('Metapopulation SIR Model with Markovian Mobility')
plt.grid(True)
plt.show()

tempo guadagnato grazie alle restrizioni

In [ ]:
for i in range(num_patches):
    print('Population #'+str(i), round(t_eval[np.argmax(Ic[i])]-t_eval[np.argmax(I[i])]), 'days')

#### verify what happens with subpop R0 < 1  
#### verify delay of epidemics when banning 80 - 99% of trips
#### compute 95% interval and median of stochastic model